# Hyperparameter tuning 

In [1]:
#essentials
import os
import numpy as np
import pandas as pd
import dataframe_image as dfi
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#tools/metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
import shap

#modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

#pandas show all columns
pd.set_option('display.max_columns', None)

#%load_ext autoreload
#%autoreload 2
#%run ./__init__

In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions, pd.options.display.max_rows

In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error


In [5]:
kp =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KenPom Team Metrics.csv")
kp_conf_rat =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KP_Conf_Ratings.csv")
models_parameters =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\model_parameters_defense.csv")


In [6]:
kp.head(2)

,Season,TeamName,TeamName 2,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
0,2019,ABILENE CHRISTIAN,Abilene Christian,66.5681,293,66.0042,254,107.1020,87,103.625,186,96.7466,43,101.539,119,2.08626,145,15.0
1,2019,AIR FORCE,Air Force,67.2794,269,66.4070,234,98.7687,274,100.431,250,105.6820,242,107.275,238,-6.84432,241,NaN


In [7]:
kp_conf_rat.head(2)

,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2023,Big 12,1,Big 12 Conference,17.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,Big Ten,2,Big Ten Conference,14.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#data_path = os.path.dirname(os.getcwd()) + '/data' + '/master_table.csv'
#master_table = pd.read_csv(data_path)
#master_table = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\final_dataset.csv")

In [9]:
#master_table.head()

In [10]:
#list(master_table.columns)

In [11]:
d = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\final_data_by_player_type.csv")
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')

#df['offense'] = df['X.Min_n1']*df['ORtg_n1']
#df['offense'] = df['X.Min_n1']*df['bpm_n1']  #box plus minus
# (stl_per_poss+blk_per_poss+Dreb%)*%Min
df['defence'] = (df['stl_per_poss_n1'] + df['blk_per_poss_n1'] + df['drb_pct_n1']) * df['X.Min_n1']

In [12]:
df[['X.Min_n1','mp_per_g','games']].describe()

,X.Min_n1,mp_per_g,games
count,11042.000000,11042.000000,11042.000000
mean,48.420766,19.833680,25.684749
std,25.011647,8.984115,9.599929
min,2.000000,1.600000,1.000000
25%,27.825000,12.100000,20.000000
50%,51.500000,20.300000,28.000000
75%,69.700000,27.400000,31.000000
max,96.500000,38.700000,144.000000


In [13]:
df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>0.1)].index)
df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>0.1)].index)


df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
idx = df.groupby(['Team', 'year'])['games'].max()
a = pd.DataFrame(idx)
ab = a.reset_index()
ab.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab, left_on=['Team', 'year'], right_on=['Team_max', 'year_max'], how='left')
idx2 = df.groupby(['Team_n1', 'year_n1'])['games_n1'].max()
a2 = pd.DataFrame(idx2)
ab2 = a2.reset_index()
ab2.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab2, left_on=['Team_n1', 'year_n1'], right_on=['Team_max', 'year_max'], how='left')

In [14]:
df = df.drop(df.loc[(df['games_max_x']<15)].index)
df = df.drop(df.loc[(df['games_max_y']<15)].index)

#df = df.drop(df.loc[(df['year']==2020) | (df['year_n1']==2020)].index)
df = df.drop(df[(df['X.Min_n1']<=10)].index)
df = df.drop(df[(df['X.Min']<=10)].index)

In [15]:
# df = df.drop(df[(df['games']<=0.05*df['games_max_x'])].index)
# df = df.drop(df[(df['games_n1']<=0.05*df['games_max_y'])].index)

In [ ]:
df = df.drop(df[(df['X.Min_n1']<=10)].index)
df = df.drop(df[(df['X.Min']<=10)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*df['ws']
df['Calc A2'] = df['X.Min']*df['bpm']
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
df['Calc A_n1'] =df['X.Min_n1']*df['ws_n1']
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

In [ ]:
df[['X.Min_n1','mp_per_g','games']].describe()

In [ ]:
#df = df[(df['X.Min_n1']>6) & (df['games']<40)  & (df['games']>7)]
#filtered_df
df[['X.Min_n1','mp_per_g','games']].describe()

In [ ]:
# Removed rows where Player Season was termed "career"
#indexrows = all_BR_data[all_BR_data['season']=='Career'].index
#all_BR_data.drop(indexrows , inplace=True)

In [ ]:
#filtered_df = df[(df['games']<40)  & (df['games']>7)]
#filtered_df[['Team', 'Player.Name', 'X.Min','season' ]].head(3)
#df_to_use = filtered_df.dropna()

In [ ]:
#filtered_df1 = filtered_df[filtered_df['mp_per_g'] > 6]
#filtered_df1
#df = filtered_df1

In [ ]:
#df = filtered_df

In [ ]:
#filtered_df[['X.Min_n1','mp_per_g','games']].describe()

In [ ]:
d[['Player.Name','season','Team','year', 'Team_n1', 'year_n1']].head(5)

In [ ]:
df[['Player.Name','season','Team','year', 'Team_n1', 'year_n1','Conf_Rating_x', 'Conf_Rating_y', 'defence']].head(5)

In [ ]:
kp.head(2)

In [ ]:
kp_conf_rat.head(2)

In [ ]:
list(df.columns)

In [ ]:
to_use= list(df.columns[(df. dtypes == 'float64') | (df. dtypes == 'int64') ])
len(to_use)
to_use

In [ ]:
df = df.replace(to_replace="NO IDEA",value=0)
df = df.replace(to_replace="---",value=0)
df = df.fillna(0)
pd.options.display.max_columns

In [ ]:
#columns = ['games','games_started','mp_per_g','fg_per_g','fga_per_g','fg2_per_g','fg2a_per_g','fg3_per_g','fg3a_per_g','ft_per_g','fta_per_g','orb_per_g','drb_per_g','trb_per_g','ast_per_g','stl_per_g','blk_per_g','tov_per_g','pf_per_g','pts_per_g','sos','mp','fg','fga','fg2','fg2a','fg3','fg3a','ft','fta','orb','drb','trb','ast','stl','blk','tov','pf','pts','fg_per_min','fga_per_min','fg2_per_min','fg2a_per_min','fg3_per_min','fg3a_per_min','ft_per_min','fta_per_min','trb_per_min','ast_per_min','stl_per_min','blk_per_min','tov_per_min','pf_per_min','pts_per_min','fg_per_poss','fga_per_poss','fg2_per_poss','fg2a_per_poss','fg3_per_poss','fg3a_per_poss','ft_per_poss','fta_per_poss','trb_per_poss','ast_per_poss','stl_per_poss','blk_per_poss','tov_per_poss','pf_per_poss','pts_per_poss','off_rtg','def_rtg','per','ts_pct','efg_pct','fg3a_per_fga_pct','fta_per_fga_pct','pprod','orb_pct','drb_pct','trb_pct','ast_pct','stl_pct','blk_pct','tov_pct','usg_pct','ows','dws','ws','ws_per_40','obpm','dbpm','bpm','year','Ht','Wt','G','S','X.Min','ORtg','X.Poss','X.Shots','eFG.','TS.','OR.','DR.','ARate','TORate','Blk.','Stl.','FC.40','FD.40','FTRate','Pct.2','X.Pct','X.Pct.1','Season_x','Tempo_x','RankTempo_x','AdjTempo_x','RankAdjTempo_x','OE_x','RankOE_x','AdjOE_x','RankAdjOE_x','DE_x','RankDE_x','AdjDE_x','RankAdjDE_x','AdjEM_x','RankAdjEM_x','seed_x','Season_y','Tempo_y','RankTempo_y','AdjTempo_y','RankAdjTempo_y','OE_y','RankOE_y','AdjOE_y','RankAdjOE_y','DE_y','RankDE_y','AdjDE_y','RankAdjDE_y','AdjEM_y','RankAdjEM_y','seed_y', 'Year_Conf_x','Conf_Rk_x','Conf_Rating_x','Year_Conf_y','Conf_Rk_y','Conf_Rating_y']
columns = ['player_type', 'games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg2_per_g', 'fg2a_per_g', 'fg3_per_g', 'fg3a_per_g', 'ft_per_g', 'fta_per_g', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'eFG.', 'TS.', 'OR.', 'DR.', 'ARate', 'TORate', 'Blk.', 'Stl.', 'FC.40', 'FD.40', 'FTRate', 'Pct.2', 'X.Pct', 'X.Pct.1', 'Season_x', 'Tempo_x', 'RankTempo_x', 'AdjTempo_x', 'RankAdjTempo_x', 'OE_x', 'RankOE_x', 'AdjOE_x', 'RankAdjOE_x', 'DE_x', 'RankDE_x', 'AdjDE_x', 'RankAdjDE_x', 'AdjEM_x', 'RankAdjEM_x', 'seed_x', 'Season_y', 'Tempo_y', 'RankTempo_y', 'AdjTempo_y', 'RankAdjTempo_y', 'OE_y', 'RankOE_y', 'AdjOE_y', 'RankAdjOE_y', 'DE_y', 'RankDE_y', 'AdjDE_y', 'RankAdjDE_y', 'AdjEM_y', 'RankAdjEM_y', 'seed_y', 'Year_Conf_x', 'Conf_Rk_x', 'Conf_Rating_x', 'Year_Conf_y', 'Conf_Rk_y', 'Conf_Rating_y', 'year_max_x', 'games_max_x', 'year_max_y', 'games_max_y', 'Team_AdjEM_dif', 'KP_Rk_Dif', 'KP_Conf_Dif', 'Calc A', 'Calc A2', 'Calc A3', 'Calc B', 'Calc C', 'Calc Overall A', 'Calc Overall A2', 'Calc Overall A3']
 


In [ ]:
master_table  = df[columns]
print(master_table.shape)

In [ ]:
y = df['defence'].to_frame()
y.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA().fit(master_table)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
pca = PCA(n_components=15)
projected= pca.fit_transform(master_table)
print(projected.shape)

In [ ]:
pca_components = pd.DataFrame(projected)

In [ ]:
pca_components.head(2)

In [ ]:
pca_components.shape

In [ ]:
# split the data with 90% in training set
X_train, X_test, y_train, y_test = train_test_split(master_table, y, random_state=0,
                                    train_size=0.9)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model_params = {
    
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators': [1,5,10,20, 25, 30, 50],
            'max_depth':[2,4,6,8,10,12]
        }
    },
    
    'naive_bayes_gaussian': {
        'model': SVR(),
        'params': {
            'C': [0.4, 0.6, 0.8, 1],
            'epsilon': [0.01, 0.02, 0.04, 0.06, 0.08, 0.1]
        }
    },
    
    'xg_boost':  {
        'model': XGBRegressor(),
        'params' : {
            'max_depth' : [2,4, 6, 8, 10, 15, 20],
            'learning_rate' : [0.1, 0.25, 0.5, 0.75],
            'subsample' : [0.5],
            'min_child_weight' : [0.1, 0.25, 0.5, 0.75]
        }
        
    }

      
}

In [ ]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    rgr =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    rgr.fit(X_train, y_train)
    print(f" model --> {model_name} done ")
    scores.append({
        'model': model_name,
        'best_score': rgr.best_score_,
        'best_params': rgr.best_params_
    })
    
df_params = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_params

In [ ]:
df_params['best_params'].iloc[2]

# Best parameters

{'learning_rate': 0.1,
 'max_depth': 2,
 'min_child_weight': 0.1,
 'subsample': 0.5}

In [ ]:
#df_params_pca = df_params

In [ ]:
df_params_pca.head()